<a href="https://colab.research.google.com/github/Arminhosseini/Justice_in_work/blob/main/justice_in_work3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['dataset.csv']))
data = df.dropna()
# data

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data['id'] = le.fit_transform(data['id'])
data['ProId'] = le.fit_transform(data['ProId'])
data['CustId'] = le.fit_transform(data['CustId'])
data['ServiceId'] = le.fit_transform(data['ServiceId'])
data['CategoryId'] = le.fit_transform(data['CategoryId'])
data['ZoneId'] = le.fit_transform(data['ZoneId'])
data['TargetProId'] = le.fit_transform(data['TargetProId'])
data['isFraud'] = le.fit_transform(data['isFraud'])
# data.head()

In [ ]:
cols = [col for col in data.columns if col not in ['id','TaskState']]
data = data[cols]

In [ ]:
cat_vars=['ProId','CustId','Rate','ServiceId','CategoryId','ZoneId']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(data[var], prefix=var)
    data1=data.join(cat_list)
    data=data1
cat_vars=['ProId','CustId','Rate','ServiceId','CategoryId','ZoneId']
data_vars=data.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]

In [ ]:
data_final=data[to_keep]
# data_final.columns.values

array(['ProCreationTimestamp', 'CustCreationTimestamp',
       'TaskCreationTimestamp', ..., 'ZoneId_32', 'ZoneId_33',
       'ZoneId_34'], dtype=object)

In [ ]:
X = data_final.loc[:, data_final.columns != 'isFraud']
y = data_final.loc[:, data_final.columns == 'isFraud']
from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns
os_data_X,os_data_y=os.fit_resample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['isFraud'])
# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of non Fraud in oversampled data",len(os_data_y[os_data_y['isFraud']==False]))
print("Number of Fraud",len(os_data_y[os_data_y['isFraud']==True]))
print("Proportion of non Fraud data in oversampled data is ",len(os_data_y[os_data_y['isFraud']==False])/len(os_data_X))
print("Proportion of Fraud data in oversampled data is ",len(os_data_y[os_data_y['isFraud']==True])/len(os_data_X))

length of oversampled data is  2636
Number of non Fraud in oversampled data 1318
Number of Fraud 1318
Proportion of non Fraud data in oversampled data is  0.5
Proportion of Fraud data in oversampled data is  0.5


In [ ]:
!pip install -U imbalanced-learn
# import imblearn
# print(imblearn.__version__)
# !pip freeze


In [ ]:
data_final_vars=data_final.columns.values.tolist()
y=['isFraud']
X=[i for i in data_final_vars if i not in y]
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
sfs = SequentialFeatureSelector(neigh, n_features_to_select=3)
sfs.fit(os_data_X, os_data_y.values.ravel())
print(sfs.get_support())

[False False False ... False False False]


In [ ]:
rightFeatures = np.array([])
for i in range(sfs.support_.shape[0]):
  if sfs.support_[i] == True:
    rightFeatures = np.append(rightFeatures, X[i])

rightFeatures

array(['Price', 'ServiceId_93', 'CategoryId_4'], dtype='<U32')

In [ ]:
X=os_data_X[rightFeatures]
y=os_data_y['isFraud']

In [ ]:
data_train, data_test, target_train, target_test = train_test_split(X,y, test_size = 0.30, random_state = 10)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
neigh = KNeighborsClassifier(n_neighbors=30)
neigh.fit(data_train, target_train)
pred = neigh.predict(data_test)
print ("KNeighbors accuracy score : ",accuracy_score(target_test, pred))

KNeighbors accuracy score :  0.606826801517067


In [ ]:
import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

In [ ]:
cols = ['ProCreationTimestamp','TaskCreationTimestamp','Price','TargetProId','ProId_41',
        'ProId_84','Rate_4','ServiceId_1','CategoryId_0','ZoneId_1','ZoneId_5']

X=os_data_X[cols]
y=os_data_y['isFraud']
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

In [ ]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))